# Neurophysiological signal processing and Network analysis

# Practical Session: Source Localization

## Intro

In this practical session, we will continue working on the data of the previous session. As a reminder, this data was recorded from healthy controls while they were performing a language task, more specifically, a word order test. In the incorrect condition, this resulted in a larger response on the central electrodes between 0.5s and 1s after the presentation of the word. This response is called the P600. Next to this, also a very clear visual response could be seen around 0.1s-0.3s after the presentation of the word.  

The goal of this practical session is to get a better idea on where in the brain this activity is generated. Therefor, we will make use of EEG source imaging (ESI) techniques. A short recap of this technique is given below.

## EEG source imaging

### What do we measure?
 
In order to generate measureable EEG signals in the brain, a large group of synchronously activated and similarly oriented neurons is needed. This is the case for pyramidal neuron cells, which are located inside the gray matter in the brain. These neurons are the main direct source of EEG signals. Because of these characteristics, the synaptic currents of these pyramidal cells can be modeled using an electrical current dipole, which has a location, intensity and orientation.

### Forward modeling and inverse solutions

ESI consists of two different parts: forward modeling and inverse solutions. In forward modeling, the goal is to create a model that shows what signals would be measured by our EEG electrodes if a specific dipole was activated with a certain intensity and orientation. This information is needed to solve the inverse problem: given the activity that we measured with the EEG electrodes, which dipoles were active?

#### Forward model

The forward model contains information about the head model and the source space, i.e. where the different dipoles are located. 

Algebraicly, we say that:

$$ \textbf{V} = \sum_{i = 1}^{N} \mathbf{L(r_i)d(r_i)} + \epsilon $$

Here, $\textbf{V} \in \mathbb{R}^{K \times 1}$ represents the measured EEG signals, with K the number of electrodes. $\textbf{L(r)} \in \mathbb{R}^{K \times 3}$ represents the lead field matrix that captures information about the headmodel properties (geometry, electrode locations, conductivities). Finally, $\textbf{d(r)} \in \mathbb{R}^{3 \times 1}$ represents the dipole source in 3 orthogonal directions on location $\textbf{r}$ inside the gray matter and N is the number of dipoles that is considered.

In order to create the lead field matrix L, both a head model and a source space is needed. The head model is constructed based on the segmentation of an anatomical (T1) MR image into different tissue types and subsequently coregistering the electrode positions on this head model. In this practicum we will work with a 3-layered head model, in which scalp, skull and brain are modeled. Different types of so-called source spaces can be used in the forward model. The source space describes the number of dipoles and their positions. In this practical session, we will consider different approaches, namely single dipole, multiple dipoles and distributed dipoles.

Different numerical methods exist to calculate the lead field matrix L in a realistic head model, e.g. BEM, FEM and FDM. As MNE has only implemented the BEM method, or boundary element method, this is the approach that will be used during this practical session. 


#### Inverse problem
As mentioned before, the forward model describes what EEG signals we would measure when a given dipole in the brain is active. However, we are interested in the inverse problem: we have measured the EEG and we want to know which sources in the brain have generated this EEG. Several techniques exist to solve the inverse problem. We can distinguish two types of techniques: 

- Equivalent current dipole (ECD) approaches, where the EEG signals are assumed to be generated by a relatively small number (<10) of focal sources. An example of this approach is to find the dipole(s) that minimize the relative residual error (RRE).
- Distributed dipoles approaches, where all possible source locations, typically around 10.000, are considered simultaneously. Examples of the distributed dipole approaches are minimum norm solution (MNE), weighted MNE, LORETA, dSPM...


### Goal of today

In this practical session, we will use each of the source spaces mentioned above to localize the P600 peak and the visual response in the brain. For this, we will use the grand-average responses that were obtained in the previous practical session. Where possible, we will compare different methods and check if similar results are found or not.

## Part 1: Import the necessary libraries and load the data

In [1]:
import mne
import os
import glob
import numpy as np

from mne.datasets import fetch_fsaverage

%matplotlib qt
import matplotlib.pyplot as plt

#You might need this if you have a Mac
#os.environ['QT_MAC_WANTS_LAYER']='1'

### Download, load and plot the head model and the source space

As no MRI images were recorded for the subjects included in the P600 experiment, during this practical session, we will use an average MR image to create an average head model. The files that you will need can be downloaded from the MNE site as follows:

(Tip: the data will be saved in the folder where you opened Jupyter Notebook. If you want to change this location, close your notebook, navigate in your terminal to the folder that you want to use and restart Jupyter Notebook there.)

In [63]:
fs_dir = fetch_fsaverage(verbose=True)
subjects_dir = os.path.dirname(fs_dir)

subject = 'fsaverage'
trans = 'fsaverage'

# -src files contain information about the source space, i.e. where the different dipoles are located
# will be used later
src_surface = os.path.join(fs_dir, 'bem', 'fsaverage-ico-5-src.fif')
src_volume = os.path.join(fs_dir, 'bem', 'fsaverage-vol-5-src.fif')

# -bem-sol files contain information about the BEM-head model that is used, i.e. the segmentation of the MR image
bem = os.path.join(fs_dir, 'bem', 'fsaverage-5120-5120-5120-bem-sol.fif')

0 files missing from root.txt in C:\Users\neu_d\mne_data\MNE-fsaverage-data
0 files missing from bem.txt in C:\Users\neu_d\mne_data\MNE-fsaverage-data\fsaverage


### Load the P600 epochs and the grand averages

We will reuse the P600-epochs that were created during the last practical session. You can either use your own obtained files, or you can load the files that were provided:

In [54]:
# Epochs
EPOCHS_DIR = './Epochs/'
files = sorted(glob.glob(EPOCHS_DIR + '*.fif'))

all_epochs = []

for file in files:
    epochs = mne.read_epochs(file, verbose = False)
    all_epochs.append(epochs)

# Grand Averages
grand_average_correct = mne.read_evokeds('./Evoked/GrandAverage-correct-ave.fif', verbose = False)[0]
grand_average_incorrect = mne.read_evokeds('./Evoked/GrandAverage-incorrect-ave.fif', verbose = False)[0]

grand_average_correct = grand_average_correct.set_eeg_reference(projection=True, verbose = False)
grand_average_incorrect = grand_average_incorrect.set_eeg_reference(projection=True, verbose = False)

### Task 1.1: Plot for grand averages of both conditions and the topographies over time of the incorrect condition 

Do this as a reminder of the results we obtained during the previous session.

In [23]:
channel = 'Pz'
grand_average_correct_pz = grand_average_correct.copy().pick([channel])
grand_average_incorrect_pz =grand_average_incorrect.copy().pick([channel])

plt.figure(figsize=(8, 5))
plt.plot(grand_average_correct_pz.times, grand_average_correct_pz.data.T, label='Correct')
plt.plot(grand_average_incorrect_pz.times, grand_average_incorrect_pz.data.T, label='Incorrect')
plt.axvline(0, color='black', linestyle='--', label='Stimulus Onset')
plt.xlabel('Time (s)')
plt.ylabel('Amplitude (µV)')
plt.legend()
plt.grid(True)
plt.show()

In [11]:
time_points = [0.1,0.3,0.6,0.9]  #random

fig,axes = plt.subplots(1, len(time_points), figsize=(15, 6))

# Loop through time points and plot topographies
for i, t in enumerate(time_points):

    grand_average_incorrect.plot_topomap(times=t, axes=axes[i],colorbar =False, show=False)
    axes[i].set_title(f'Incorrect: {t:.1f}s')

# Add overall figure title
fig.suptitle("Topographies of Evoked Responses at Different Time Points", fontsize=16)
plt.show()

### Task 1.2: Crop the grand-averages

As mentioned in the previous practical session, the evoked potentials plotted before do not only show the P600, but also the visual responses to the words. Crop the evoked potentials to one time point so that we can use only the relevant parts if the incorrect response to localize 1) the P600 peak and 2) the visual response. Find the exact time of the peak for the P600 response. For the visual response, you can use time-point 0.1s.

In [55]:
visual_crop = grand_average_incorrect.copy().crop(0.1,0.1)

In [59]:
P600_peaktime = grand_average_incorrect.get_peak(tmin=0.5,tmax = 0.8)[1]

P600_crop = grand_average_incorrect.copy().crop(P600_peaktime,P600_peaktime)

## Part 2: Single Dipole Fit

We will start by fitting a single dipole to the evoked data. Look at [this](https://mne.tools/stable/auto_tutorials/source-modeling/plot_dipole_fit.html?highlight=single%20dipole) tutorial for help. 

Tip: during this practical session, a lot of functions will print long comments. Use 'verbose = False' as an extra option for these functions to avoid this.

### Task 2.1: Calculate the covariance matrix

Notice that a noise covariance matrix is needed to do this. Use the list of epochs that was loaded before to create/estimate this matrix. Make sure to specify that only the pre-stimulus period needs to be used to estimate this matrix. 

***Question:*** Why do you think that we use the pre-stimulus period?

**Answers**
After stimulus there's brain activity that's event related, and the assumptions was pre-stimulus is the baseline where 'no active brain activities'.

In [21]:
noise_cov = mne.compute_covariance(all_epochs, tmax=0.0, verbose=False)

### Task 2.2: Fit a single dipole to the P600 peak and the visual response.

Plot the obtained dipole both in 3D (use MNE functions for this) and in 2D. For the latter, the function 'plot_single_dipole(dipole, subjects_dir, subject)' was included.

Note: subjects_dir and subject were defined at the beginning of this notebook.

***Question:*** Where are the dipoles located? Do these locations make sense/does they correspond with what you expected? Explain.

In [ ]:
# Use the following commands to install some packages if needed

# !pip install nilearn

In [14]:
from nilearn import plotting

# They made the plot_markers-function so that you cannot plot 1 dipole, solution: plot the same dipole twice...
def plot_single_dipole(dipole, subjects_dir, subject):
    
    trans = mne.read_trans(os.path.join(subjects_dir, subject, 'bem', 'fsaverage-trans.fif'))
    
    positions = np.zeros((2,3))
    positions[0,:] = mne.head_to_mri(dipole.pos, mri_head_t=trans ,subject=subject, subjects_dir=subjects_dir)
    positions[1,:] = positions[0,:]
    node_values = np.ones(shape = (2,1))
    
    fig = plotting.plot_markers(node_values, positions, title='Dipole locations', node_size=30, colorbar = False)
    return fig

In [ ]:
# Fit a dipole
dip = mne.fit_dipole(P600_crop, noise_cov, bem, trans)[0]
# 2d
fig = plot_single_dipole(dip, subjects_dir, subject)

MRI transform     : d:\Users\neu_d\anaconda3\envs\neuro\Lib\site-packages\mne\data\fsaverage\fsaverage-trans.fif
Head origin       :   -1.4    7.9   51.3 mm rad =   80.0 mm.
Guess grid        :   20.0 mm
Guess mindist     :    5.0 mm
Guess exclude     :   20.0 mm
Using normal MEG coil definitions.
Noise covariance  : <Covariance | kind : full, shape : (31, 31), range : [-1.3e-11, +6.3e-11], n_samples : 528952>

Coordinate transformation: MRI (surface RAS) -> head
    0.999994 0.003552 0.000202      -1.76 mm
    -0.003558 0.998389 0.056626      31.09 mm
    -0.000001 -0.056626 0.998395      39.60 mm
    0.000000 0.000000 0.000000       1.00
Coordinate transformation: MEG device -> head
    1.000000 0.000000 0.000000       0.00 mm
    0.000000 1.000000 0.000000       0.00 mm
    0.000000 0.000000 1.000000       0.00 mm
    0.000000 0.000000 0.000000       1.00
0 bad channels total
Read  31 EEG channels from info
Head coordinate coil definitions created.
Decomposing the sensor noise covar

In [ ]:
fig = dip.plot_locations(trans, subjects_dir=subjects_dir, subject=subject)


## Part 3: Calculate the leadfield matrix

In part 1 of this notebook, a head space (called bem) and two different source spaces (src_surface and src_volume) were loaded. We will use these files to create the leadfield matrix.

### Task 3.1: Plot the head model and the source spaces

Look at the difference between the two source spaces that were given using the [mne.viz.plot_bem()](https://mne.tools/stable/generated/mne.viz.plot_bem.html?highlight=plot_bem#mne.viz.plot_bem) command. 

***Question:*** What is the difference between both source spaces?

***Question:*** How many dipoles are used in both source spaces?

In [ ]:
# Plot the head model (BEM) with src surface
fig = mne.viz.plot_bem(subject=subject, subjects_dir=subjects_dir, show=True,src= src_surface)

Using surface: C:\Users\neu_d\mne_data\MNE-fsaverage-data\fsaverage\bem\inner_skull.surf
Using surface: C:\Users\neu_d\mne_data\MNE-fsaverage-data\fsaverage\bem\outer_skull.surf
Using surface: C:\Users\neu_d\mne_data\MNE-fsaverage-data\fsaverage\bem\outer_skin.surf
    Reading a source space...
    [done]
    Reading a source space...
    [done]
    2 source spaces read


In [ ]:
# Plot the head model (BEM) with src_volume
fig = mne.viz.plot_bem(subject=subject, subjects_dir=subjects_dir, show=True,src= src_volume)

Using surface: C:\Users\neu_d\mne_data\MNE-fsaverage-data\fsaverage\bem\inner_skull.surf
Using surface: C:\Users\neu_d\mne_data\MNE-fsaverage-data\fsaverage\bem\outer_skull.surf
Using surface: C:\Users\neu_d\mne_data\MNE-fsaverage-data\fsaverage\bem\outer_skin.surf
    Reading a source space...
    [done]
    1 source spaces read


In [88]:
num_dipoles_surface = len(src_surface) 

num_dipoles_volume = len(src_volume) 

print(f"Number of dipoles in surface source space: {num_dipoles_surface}")
print(f"Number of dipoles in volume source space: {num_dipoles_volume}")


Number of dipoles in surface source space: 80
Number of dipoles in volume source space: 80


### Task 3.2: Calculate the forward solutions for both source spaces

Based on a source space and the head model, a leadfield matrix (also called the forward solution) can be calculated by using the function [mne.make_forward_solution()](https://mne.tools/stable/generated/mne.make_forward_solution.html).

***Question:*** Why do we need the source space and the head model to create the forward solution? To answer this question, think about what information both represent.

***Question:*** Explain shortly, in your own words, what the leadfield matrix represents.

***Question:*** Check if the shape of the leadfield matrices are correct. [This](https://mne.tools/stable/auto_tutorials/source-modeling/plot_forward.html#sphx-glr-auto-tutorials-source-modeling-plot-forward-py) tutorial shows how you can do this.



In [85]:
# Create forward solution
fwd_surface = mne.make_forward_solution('./Evoked/GrandAverage-incorrect-ave.fif',trans, src_surface, bem)

# Check the shape of the leadfield matrix
print(fwd_surface['sol']['data'].shape)


Source space          : C:\Users\neu_d\mne_data\MNE-fsaverage-data\fsaverage\bem\fsaverage-ico-5-src.fif
MRI -> head transform : d:\Users\neu_d\anaconda3\envs\neuro\Lib\site-packages\mne\data\fsaverage\fsaverage-trans.fif
Measurement data      : GrandAverage-incorrect-ave.fif
Conductor model   : C:\Users\neu_d\mne_data\MNE-fsaverage-data\fsaverage\bem\fsaverage-5120-5120-5120-bem-sol.fif
Accurate field computations
Do computations in head coordinates
Free source orientations

Reading C:\Users\neu_d\mne_data\MNE-fsaverage-data\fsaverage\bem\fsaverage-ico-5-src.fif...
Read 2 source spaces a total of 20484 active source locations

Coordinate transformation: MRI (surface RAS) -> head
    0.999994 0.003552 0.000202      -1.76 mm
    -0.003558 0.998389 0.056626      31.09 mm
    -0.000001 -0.056626 0.998395      39.60 mm
    0.000000 0.000000 0.000000       1.00

Read  31 EEG channels from info
Head coordinate coil definitions created.
Source spaces are now in head coordinates.

Setting up t

In [86]:
fwd_volume = mne.make_forward_solution('./Evoked/GrandAverage-incorrect-ave.fif',trans, src_volume, bem)
print(fwd_volume['sol']['data'].shape)

Source space          : C:\Users\neu_d\mne_data\MNE-fsaverage-data\fsaverage\bem\fsaverage-vol-5-src.fif
MRI -> head transform : d:\Users\neu_d\anaconda3\envs\neuro\Lib\site-packages\mne\data\fsaverage\fsaverage-trans.fif
Measurement data      : GrandAverage-incorrect-ave.fif
Conductor model   : C:\Users\neu_d\mne_data\MNE-fsaverage-data\fsaverage\bem\fsaverage-5120-5120-5120-bem-sol.fif
Accurate field computations
Do computations in head coordinates
Free source orientations

Reading C:\Users\neu_d\mne_data\MNE-fsaverage-data\fsaverage\bem\fsaverage-vol-5-src.fif...
Read 1 source spaces a total of 14629 active source locations

Coordinate transformation: MRI (surface RAS) -> head
    0.999994 0.003552 0.000202      -1.76 mm
    -0.003558 0.998389 0.056626      31.09 mm
    -0.000001 -0.056626 0.998395      39.60 mm
    0.000000 0.000000 0.000000       1.00

Read  31 EEG channels from info
Head coordinate coil definitions created.
Source spaces are now in head coordinates.

Setting up t

## Part 4: Multiple Dipole Fit - RAP MUSIC

The multiple signal classification (MUSIC) algorithm can be used to locate multiple asynchronous dipolar sources. We will not go over the mathematical details, but for more details you can check [this article](https://journals.lww.com/clinicalneurophys/Fulltext/1999/05000/EEG_Source_Localization_and_Imaging_Using_Multiple.4.aspx?casa_token=S276hn0JPDoAAAAA:j55kYq5jl6k99xdjxvxa6p5gnB_vwphUms-3dGX6X_vr2dyAFng_b_2mRZVnbkb_yhB81LAunahRYdWMeibk-Zj9Ig91ZDCS) from Mosher, John C et al. (1999).

The algoritm is implemented in MNE as mne.beamformer.rap_music. Use this function to obtain 5 dipoles for both the P600 and the visual response. As before, plot the dipoles both in 3D (using MNE-functions) and in 2D (using the plot_dipoles-function below).

In [84]:
def plot_dipoles(dipoles, subjects_dir, subject):
    
    trans = mne.read_trans(os.path.join(subjects_dir, subject, 'bem', 'fsaverage-trans.fif'))
    
    positions = np.zeros((len(dipoles),3))
    for index, dip in enumerate(dipoles):
        mri_pos = mne.head_to_mri(dip.pos, mri_head_t=trans ,subject=subject, subjects_dir=subjects_dir)
        positions[index,:] = mri_pos
    
    #node_values=np.ones(shape = (len(dipoles),1))
    node_values=np.arange(0, len(dipoles))
    
    plotting.plot_markers(node_values,positions, title='Dipole locations', node_size=30, colorbar = False)

### Task 4.1: P600, surface and volume source space

***Question:*** When I showed the obtained result of the P600 localisation to our colleagues in the BrainComm research group, they were quite enthousiastic about the dipole close to the left inferior frontal gyrus. Why is this?

In [94]:
dipoles_P600surface, residual_P600surface = mne.beamformer.rap_music(P600_crop, fwd_surface, noise_cov, n_dipoles=5, return_residual=True, verbose=False)

fig = plot_dipoles(dipoles_P600surface, subjects_dir=subjects_dir, subject=subject)


In [97]:
fig = mne.viz.plot_dipole_locations(dipoles_P600surface, trans, subjects_dir=subjects_dir)

TypeError: join() argument must be str, bytes, or os.PathLike object, not 'NoneType'

In [ ]:
dipoles_P600volumn, residual_P600volumn = mne.beamformer.rap_music(P600_crop, fwd_volume, noise_cov, n_dipoles=5, return_residual=True, verbose=False)

fig = plot_dipoles(dipoles_P600volumn, subjects_dir=subjects_dir, subject=subject)

### Task 4.2: Visual response, surface and volume source space

## Distributed Dipole Fit - sLORETA

Finally, we will use the sLORETA-method to reconstruct the brain activity from the evoked potential. ***Use the entire time-course of the evoked potential now instead of the cropped version.*** Take a look at [this](https://mne.tools/stable/auto_tutorials/source-modeling/plot_mne_dspm_source_localization.html#sphx-glr-auto-tutorials-source-modeling-plot-mne-dspm-source-localization-py) tutorial to help you. 

After applying the inverse solution to the data, you should obtain time-series data for each dipole included in the source space. Plot these time-series using the stc.plot() function and see how the brain activity changes over time. 

***Questions:*** Answer the following questions by including screenshots of the plots that show me the answer in your ppt. Where is the activity located around the P600 peak? Where is the visual response located? If it is difficult to locate the activity, play around with the limits of the colorbars. 

In [19]:
# Use the following commands to install/update some packages if needed

!pip install spyder -U
!pip install pyvista -U
!pip install ipyvtklink  
!pip install pyvistaqt
!pip install "PyQt5>=5.10,<5.14"

  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
   ---------------------------------------- 0.0/14.3 MB ? eta -:--:--
   ---- ----------------------------------- 1.6/14.3 MB 9.3 MB/s eta 0:00:02
   ------------- -------------------------- 5.0/14.3 MB 13.1 MB/s eta 0:00:01
   ------------------------ --------------- 8.7/14.3 MB 14.9 MB/s eta 0:00:01
   ---------------------------------- ----- 12.3/14.3 MB 15.4 MB/s eta 0:00:01
   ---------------------------------------- 14.3/14.3 MB 14.7 MB/s eta 0:00:00
   ---------------------------------------- 0.0/826.7 kB ? eta -:--:--
   ---------------------------------------- 826.7/826.7 kB 9.1 MB/s eta 0:00:00
   ---------------------------------------- 0.0/6.9 MB ? eta -:--:--
   ------------ --------------------------- 2.1/6.9 MB 10.7 MB/s eta 0:00:01
   ---------------------- ------------

ERROR: Could not find a version that satisfies the requirement PyQt5<5.14,>=5.10 (from versions: 5.14.0, 5.14.1, 5.14.2, 5.15.0, 5.15.1, 5.15.2, 5.15.3, 5.15.4, 5.15.5, 5.15.6, 5.15.7, 5.15.8, 5.15.9, 5.15.10, 5.15.11)
ERROR: No matching distribution found for PyQt5<5.14,>=5.10


In [ ]:
%matplotlib qt

## Extra: Distributed Dipole fit using different methods, e.g. dSPM